## S3 Python API Example Code

#### Boto3 is used as the S3 API module

In [ ]:
import json
import boto3
import boto3.session
import random
import string

#### Set the endpoint and generate a unique bucket name

In [ ]:
endpoint = 'https://webscaledemo.netapp.com:8082'
bucket_name = 'tmp-bucket-' + ''.join([random.choice(string.ascii_lowercase) for i in range(4)])
print("Using bucket: " + bucket_name)

#### Provide your own access credentials

In [ ]:
access_key = '<your access key here>'
secret_key = '<your secret key here>'

**Do not use the example below in production - disabling SSL verification is discouraged!**  
When using a self-signed certificate, make sure to pass it into the constructor:  
```s3 = session.resource(service_name='s3', endpoint_url=endpoint, verify='server_cert.pem')```

In [ ]:
session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key)
s3 = session.resource(service_name='s3', endpoint_url=endpoint, verify=False)
client = s3.meta.client

### Bucket related operations

#### Create new bucket for S3 account

In [ ]:
bucket = s3.Bucket(bucket_name)
bucket.create()

#### List all buckets for S3 account

In [ ]:
bucket.wait_until_exists()
for bucket in s3.buckets.all():
    print(bucket.name)

#### Enable bucket versioning

In [ ]:
bucket.Versioning().enable()

#### Get Bucket Versioning status

In [ ]:
print (bucket.Versioning().status)

### Object related operations

#### Put a new object to a bucket

In [ ]:
obj = s3.Object(bucket.name, 'my-key')
obj.put(Body='This is my object\'s data',
        Metadata={'customerid': '1234', 'location': 'germany'},
        ServerSideEncryption='AES256')

#### Copy an existing object

In [ ]:
obj.wait_until_exists()
copied_obj = s3.Object(bucket.name, 'my-copied-key')
copied_obj.copy_from(CopySource={ 'Bucket': obj.bucket_name, 'Key': obj.key })

#### Get object from bucket

In [ ]:
response = obj.get()
data = response['Body'].read()
metadata = response['Metadata']
print("Data: %s // Metadata: %s" % (data, metadata))

#### List all objects for a bucket

In [ ]:
for obj in bucket.objects.all():
    print(obj.key)

#### Generate a pre-signed URL (only possible via client, not directly via Object object)

In [ ]:
url = client.generate_presigned_url('get_object', {'Bucket': bucket.name, 'Key': 'my-key'}, ExpiresIn=3600)
print("Pre-signed URL: %s" % (url))

#### Delete all objects and versions from the bucket

In [ ]:
for obj in bucket.object_versions.all():
    obj.delete()

Create a bucket policy  
*Note the StorageGRID Webscale specific Resource URN*

In [ ]:
policy = {
    "Statement": [
        {
            "Sid":"AddPerm",
            "Effect":"Allow",
            "Principal": "*",
            "Action":["s3:ListBucket"],
            "Resource":["urn:sgws:s3:::" + bucket.name]
        }
    ]
}

Set a new policy allowing anyone to list the contents of the bucket

In [ ]:
bucket.Policy().put(Policy=json.dumps(policy))

#### Delete bucket

In [ ]:
bucket.delete()